## 0.0 read data

In [1]:
import numpy as np
from numpy import linalg
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
import time
from datetime import *
import copy

In [84]:
df_origin = pd.read_csv('./data/train.csv')
label = pd.read_csv('./data/train_label.csv')
df_test = pd.read_csv('./data/test.csv')

In [85]:
df_train = df_origin.loc[:74842, :]
df_val = df_origin.loc[74843:, :]

## concate

In [3]:
y_isCan = df_origin['is_canceled']
y_adr = df_origin['adr']
df_origin = df_origin.drop(columns=['reservation_status', 'reservation_status_date', 'is_canceled', 'adr'])

In [4]:
concate = pd.concat([df_origin,df_test],axis=0).reset_index(drop = True)

In [5]:
concate.isna().sum()

ID                                     0
hotel                                  0
lead_time                              0
arrival_date_year                      0
arrival_date_month                     0
arrival_date_week_number               0
arrival_date_day_of_month              0
stays_in_weekend_nights                0
stays_in_week_nights                   0
adults                                 0
children                               4
babies                                 0
meal                                   0
country                              488
market_segment                         0
distribution_channel                   0
is_repeated_guest                      0
previous_cancellations                 0
previous_bookings_not_canceled         0
reserved_room_type                     0
assigned_room_type                     0
booking_changes                        0
deposit_type                           0
agent                              16340
company         

### dropna

In [6]:
concate = concate.drop(columns=['country', 'agent', 'company'])

### country填median

In [7]:
concate = concate.fillna(concate.children.median())

In [8]:
concate['arrival_date_month'] = concate['arrival_date_month'].map({'January':1, 'February': 2, 'March':3, 'April':4, 'May':5, 'June':6, 'July': 7, 'August':8, 'September':9, 'October': 10, 'November': 11, 'December':12})
concate.arrival_date_month = concate.arrival_date_month.astype(int)

### 加星期幾

In [9]:
def dateToWeekDay(df):
    year = df.arrival_date_year
    month = df.arrival_date_month
    day = df.arrival_date_day_of_month
    n = len(df)
    weekDays = [0 for i in range(n)]
    for i in range(n):
        weekDays[i] = date(year[i], month[i], day[i]).weekday()   
    # return 1~7
    df1 = copy.deepcopy(df)
#     df1 = df1.rename(columns = {'arrival_date_year': 'year', 'arrival_date_month': 'month'})
    df1.insert(6,'weekDay', weekDays)
    df1 = pd.get_dummies(df1, columns = ['weekDay'])
    print(df1.columns)
    df1 = df1.drop(columns = ['weekDay_6']) # remove last one
    return df1;

In [10]:
concate = dateToWeekDay(concate)

Index(['ID', 'hotel', 'lead_time', 'arrival_date_year', 'arrival_date_month',
       'arrival_date_week_number', 'arrival_date_day_of_month',
       'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children',
       'babies', 'meal', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type',
       'days_in_waiting_list', 'customer_type', 'required_car_parking_spaces',
       'total_of_special_requests', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'weekDay_6'],
      dtype='object')


### getdummy

In [11]:
# Creating dummy variables:
concate = pd.get_dummies(concate, columns=['hotel', 'arrival_date_month', 'meal', 'market_segment', 'distribution_channel', \
                                 'reserved_room_type', 'assigned_room_type', 'deposit_type', 'customer_type'])
print(concate.columns)

Index(['ID', 'lead_time', 'arrival_date_year', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'booking_changes',
       'days_in_waiting_list', 'required_car_parking_spaces',
       'total_of_special_requests', 'weekDay_0', 'weekDay_1', 'weekDay_2',
       'weekDay_3', 'weekDay_4', 'weekDay_5', 'hotel_City Hotel',
       'hotel_Resort Hotel', 'arrival_date_month_1', 'arrival_date_month_2',
       'arrival_date_month_3', 'arrival_date_month_4', 'arrival_date_month_5',
       'arrival_date_month_6', 'arrival_date_month_7', 'arrival_date_month_8',
       'arrival_date_month_9', 'arrival_date_month_10',
       'arrival_date_month_11', 'arrival_date_month_12', 'meal_BB', 'meal_FB',
       'meal_HB', 'meal_SC', 'meal_Undefined', 'market_segment_Aviation',
       'market_segment_Complementary',

In [12]:
concate = concate.drop(columns=['hotel_Resort Hotel', 'arrival_date_month_9', 'meal_Undefined', 'market_segment_Undefined', 'distribution_channel_Undefined', 'reserved_room_type_P', 'assigned_room_type_P',  'deposit_type_Refundable', 'customer_type_Transient-Party'])

In [13]:
concate.isna().sum()

ID                           0
lead_time                    0
arrival_date_year            0
arrival_date_week_number     0
arrival_date_day_of_month    0
                            ..
deposit_type_No Deposit      0
deposit_type_Non Refund      0
customer_type_Contract       0
customer_type_Group          0
customer_type_Transient      0
Length: 75, dtype: int64

### 切成train, validate, test

In [14]:
# y_isCan, y_adr 要切train, val
y_train_isCan = y_isCan.loc[:74842]
y_train_adr = y_adr.loc[:74842]
y_val_isCan = y_isCan.loc[74843:]
y_val_adr = y_isCan.loc[74843:]

#concate 要切train, val, test
X_train = concate.loc[:74842, :]
X_val = concate.loc[74843:91530, :]
X_test = concate.loc[91530:, :]

In [15]:
print(len(y_train_isCan))
print(len(y_train_adr))
print(len(y_val_isCan))
print(len(y_val_adr))
print(len(X_train))
print(len(X_val))
print(len(X_test))

74843
74843
16688
16688
74843
16688
27860


## Basic analysis

#### Target: "is_canceled"

Steps:

1. Predict "is_canceled" by attributes in each tuple

2. Multiply predictions by "ADR"

3. Group by date

4. Predict classification of the daily revenue to the labels on $y$.


In [21]:
df_dum.describe()

,ID,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
count,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,...,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000,91531.000000
mean,45765.000000,0.342299,0.357912,96.286843,0.899291,2.448438,1.835761,0.090964,0.008369,0.033890,...,0.020102,0.005714,0.003256,0.002546,0.000011,0.869957,0.128568,0.037834,0.004873,0.723646
std,26422.868082,0.474482,0.479389,105.454726,1.004555,1.935080,0.605828,0.373909,0.101749,0.180947,...,0.140352,0.075375,0.056966,0.050390,0.003305,0.336353,0.334724,0.190796,0.069635,0.447197
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22882.500000,0.000000,0.000000,15.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,45765.000000,0.000000,0.000000,58.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,68647.500000,1.000000,1.000000,145.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,91530.000000,1.000000,1.000000,737.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


#### Building X and y

#### Splitting train/validation
(isCan/adr)

Split data at index 74843

:74842 -> train

74843: -> val

In [30]:
X_train

,ID,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
0,0,1,342,0,0,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
1,1,0,257,0,2,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,2,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3,3,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,4,0,257,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74838,74838,1,1,0,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74839,74839,1,1,0,2,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74840,74840,0,138,0,3,1,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74841,74841,0,12,0,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [31]:
X_val

,ID,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,...,assigned_room_type_G,assigned_room_type_H,assigned_room_type_I,assigned_room_type_K,assigned_room_type_L,deposit_type_No Deposit,deposit_type_Non Refund,customer_type_Contract,customer_type_Group,customer_type_Transient
74843,74843,0,290,1,3,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
74844,74844,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74845,74845,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74846,74846,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
74847,74847,0,510,0,2,2,0.0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91526,91526,1,19,0,2,2,2.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91527,91527,1,28,0,2,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91528,91528,1,2,0,1,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
91529,91529,1,30,3,7,2,0.0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [32]:
y_train_isCan.value_counts(normalize=True)

0    0.638163
1    0.361837
Name: is_canceled, dtype: float64

The cancel rate is around 35%.

In [33]:
y_train_adr.value_counts()

0.000000      75
47.000000      5
31.000000      5
42.000000      5
16.000000      4
              ..
139.827646     1
79.667069      1
115.188575     1
151.865524     1
50.842307      1
Name: adr, Length: 74732, dtype: int64

### Computing accuracy 

In [23]:
from sklearn.metrics import accuracy_score
def computeAccuracy(X, y, y_result): #x: test revenue, y: test label
    print("Accuracy =", accuracy_score(y, y_result))
    print("Number of mislabeled points out of a total %d points : %d"% (X.shape[0],(y != y_result).sum()))
    print("E_val = ", (y != y_result).sum() / X.shape[0])
    
from sklearn.metrics import r2_score
def calculateRsquare(X, y, y_result): #x: test revenue, y: test label
    print("R square =", r2_score(y, y_result))

## Predict is_can

### KNN

In [55]:
def trainKNN(X_train_df, y_train, k, X_val_df, y_val):
    print("Pre-processing...")
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_df[['ID', 'lead_time', 'previous_cancellations', 'days_in_waiting_list', 'deposit_type_Non Refund']]
#     X_train_feat = X_train_feat.to_numpy()
#     X_train_feat = X_train_df.to_numpy()
    X_train_feat = X_train_df
    knn = KNeighborsClassifier(n_neighbors = k)
    print("Fitting...")
    knn_best = knn.fit(X_train_feat, y_train)
#     X_val_feat = X_val_df[['ID', 'lead_time', 'previous_cancellations', \
#                             'previous_bookings_not_canceled', 'booking_changes', \
#                             'days_in_waiting_list', 'required_car_parking_spaces', \
#                             'total_of_special_requests', 'market_segment_Groups', 'deposit_type_Non Refund']]
#     X_val_feat = X_val_df[['distribution_channel_GDS', 'reserved_room_type_B', 'reserved_room_type_L', 'assigned_room_type_B', 'assigned_room_type_L']]
#     X_val_feat = X_val_feat.to_numpy()
    X_val_feat = X_val_df
    print("Predicting...")
    y_pred = knn_best.predict(X_val_feat)
#     print("Calculating accuracy...")
#     sc = knn_best.score(X_val_feat, y_val, sample_weight=None)
    return y_pred, sc;

In [37]:
y_pred, sc = trainKNN(X_train, y_train_isCan, 5, X_val, y_val_isCan)

Pre-processing...
Fitting...
Predicting...
Calculating accuracy...


In [39]:
sc

0.6679050814956855

In [40]:
computeAccuracy(X_val, y_val_isCan, y_pred)

Accuracy = 0.6679050814956855
Number of mislabeled points out of a total 16688 points : 5542
E_val =  0.3320949185043145


### Desicion Stump

In [57]:
from sklearn import tree

def trainDecisionStump(X_train, y_train, X_val):
    clf = tree.DecisionTreeClassifier().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

def trainDecisionStumpRegressor(X_train, y_train, X_val):
    clf = tree.DecisionTreeRegressor(max_depth=10).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [42]:
y_result = trainDecisionStump(X_train, y_train_isCan, X_val)

In [43]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7062559923298178
Number of mislabeled points out of a total 16688 points : 4902
E_val =  0.29374400767018216


### Random Forest Classification

In [29]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

def trainRFC(X, y):
    rfr = RandomForestClassifier(random_state=False)
    rfr.fit(X, y)
    y_result = rfr.predict(X)
    return y_result;

In [83]:
y_result = trainRFC(X_train, y_train_isCan)
computeAccuracy(X_train, y_train_isCan, y_pred)

Accuracy = 0.9999732773940114
Number of mislabeled points out of a total 74843 points : 2
E_val =  2.6722605988536002e-05


#### tree isCan

In [44]:
y_result = trainDecisionStump(X_train_tree, y_train_tree_isCan, X_val_tree)

NameError: name 'X_train_tree' is not defined

In [263]:
computeAccuracy(X_val_tree, y_val_tree_isCan, y_result)

Accuracy = 0.652984180249281
Number of mislabeled points out of a total 16688 points : 5791
E_val =  0.3470158197507191


#### tree adr

In [316]:
y_result = trainDecisionStumpRegressor(X_train_tree, y_train_tree_adr, X_val_tree)

In [317]:
calculateRsquare(X_val_tree, y_val_tree_adr, y_result)

R square = 0.4756839274931203


### Logistic Regression

使用 L2 regularization, 搭配上'sag'solver

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


### solver: lbfgs, newton-cg, sag
def trainLogisticRegression(X_train, y_train, X_val):
    clf = LogisticRegression(random_state=0, penalty='l2', solver='newton-cg', multi_class='multinomial').fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [118]:
y_result = trainLogisticRegression(X_train, y_train_isCan, X_val)

C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:477: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\scipy\optimize\linesearch.py:327: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [119]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7870925215723873
Number of mislabeled points out of a total 16688 points : 3553
E_val =  0.21290747842761265


### SVM

In [ ]:
import numpy as np
import sys
sys.path.insert(1, '.')
from svmutil import *

In [ ]:
def toSVMdata(X, y): 
    N, d = X_train.shape
    X_svm = []
    for n in range(N):
        X_svm.append({})
        for i in range(d):
            X_svm[n].update({i: X_train[n,i]}) 
    y_svm = list(y)
    return X_svm, y_svm

In [ ]:
def SVM (Xtrain, ytrain, Xval, yval):
    bestaccu = 0
    for i in range(-2, 3):
        c = pow(10, i)
        # turn into SVM data form
        Xval, yval = toSVMdata(Xval, yval)
        Xtrain, ytrain = toSVMdata(Xtrain, ytrain)
        # train and prediction
        model = svm_train(ytrain, Xtrain, '-s 0 -t 0 -c {C}'.format(C = c))
        print("with C =", c, ",", end = "")
        p_label, p_acc, p_val = svm_predict(yval, Xval, model)
        if (p_acc[0] > bestaccu):
            bestaccu = p_acc[0]
    bestaccu /= 100
    return bestaccu

In [ ]:
a = X_train.to_numpy()
b = y_train_isCan.to_numpy()
print(type(a))
print(type(b))
X_train_svm, y_train_svm = toSVMdata(X_train.to_numpy(), y_train_isCan.to_numpy())
X_val_svm, y_val_svm = toSVMdata(X_val, y_val_isCan)
accuracy = SVM(X_train_svm, y_train_svm, X_val_svm, y_val_svm)
print("E_val =", 1 - accuracy)

### GussianNB

In [80]:
from sklearn.naive_bayes import GaussianNB

def trainGussianNB(X_train, y_train, X_val):
    clf = GaussianNB().fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [84]:
y_result = trainGussianNB(X_train, y_train_isCan, X_val)

In [85]:
computeAccuracy(X_val, y_val_isCan, y_result)

Accuracy = 0.7596476510067114
Number of mislabeled points out of a total 16688 points : 4011
E_val =  0.24035234899328858


## Predict adr

### Neural Network

In [45]:


# def trainNeural(X_train, y_train, X_val):
#     clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1).fit(X_train, y_train)
#     y_result = clf.predict(X_val)
#     return(y_result)

In [3]:
from sklearn.neural_network import MLPRegressor

def trainNeural(X_train, y_train, X_val):
    clf = MLPRegressor(random_state=1, max_iter=500).fit(X_train, y_train)
    y_result = clf.predict(X_val)
    return(y_result)

In [1]:
y_result = trainNeural(X_train, y_train_adr, X_val)

NameError: name 'trainNeural' is not defined

In [2]:
calculateRsquare(X_val, y_val_adr, y_result)

NameError: name 'calculateRsquare' is not defined

### Linear Regression

In [22]:
from sklearn.linear_model import LinearRegression

def trainLinearRegression(X_train_adr, y_train_adr, X_val_adr):
    clf = LinearRegression().fit(X_train_adr, y_train_adr)
    y_result = clf.predict(X_val_adr)
    return(y_result)

In [23]:
y_result = trainLinearRegression(X_train, y_train_adr, X_val)

In [24]:
calculateRsquare(X_val, y_val_adr, y_result)

R square = 0.4406583693111038


### Random Forest Regression

In [27]:
def trainRFR(X, y):
    rfr = RandomForestRegressor(random_state=False)
    rfr.fit(X, y)
    y_result = rfr.predict(X)
    return y_result;

In [31]:
y_result = trainRFR(X_train, y_train_adr)
calculateRsquare(X_train, y_train_adr, y_result)

R square = 0.9669126829210924


### AdaboostClassifier (for is_canceled)

from sklearn.ensemble import AdaBoostClassifier
from sklearn.datasets import make_classification


def AdaboostClassifier(x, y):
    clf = AdaBoostClassifier(n_estimators=100, random_state=0, algorithm='SAMME')
    clf.fit(x, y) 
    y_result = clf.predict(x)
    return(y_result)
    

### Calculate predicted daily revenue

In [86]:
def calculateDateRevenue(df):
    df1 = df[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "adr", "is_canceled", "stays_in_weekend_nights", "stays_in_week_nights"]]
    df1["revenue"] = (1 - df1["is_canceled"]) * df1["adr"] * (df1["stays_in_week_nights"] + df1["stays_in_weekend_nights"])
    df2 = df1[["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month", "revenue"]]
    df3 = df2.groupby(by=["arrival_date_year", "arrival_date_month", "arrival_date_day_of_month"]).sum()
#     print(df3)
    x = df3.to_numpy()
    return(x) # one column: revenue

## Revenue to label 

In [87]:
X_train_revenue = calculateDateRevenue(df_train)
print(len(X_train_revenue))
X_val_revenue = calculateDateRevenue(df_val)
print(len(X_val_revenue))

519
121


/Users/joannehuang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [88]:
label["ID"] = label.index
label_train = label.loc[:518, :]
label_val = label.loc[519:, :]

In [89]:
y_train_label = label_train["label"].to_numpy()
print(len(y_train_label))
y_val_label = label_val["label"].to_numpy()
print(len(y_val_label))

519
121


### KNN

In [50]:
y_result = trainKNN(X_train_revenue, y_train_label, 5, X_val_revenue, y_val_label)

Pre-processing...
Fitting...
Predicting...
Calculating accuracy...


In [54]:
len(y_result)

2

In [52]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

ValueError: Found input variables with inconsistent numbers of samples: [121, 2]

### Desicion Stump 

In [90]:
print(len(X_train_revenue))
print(len(y_train_label))
print(len(X_val_revenue))
print(len(y_val_label))
print(len(y_result))

519
519
121
121
74843


In [91]:
y_result = trainDecisionStump(X_train_revenue, y_train_label, X_val_revenue)

In [92]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.15702479338842976
Number of mislabeled points out of a total 121 points : 102
E_val =  0.8429752066115702


### Logistic Regression

In [77]:
y_result = trainLogisticRegression(X_train_revenue, y_train_label, X_val_revenue)

/Users/joannehuang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/joannehuang/anaconda3/lib/python3.6/site-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/Users/joannehuang/anaconda3/lib/python3.6/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [78]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.30578512396694213
Number of mislabeled points out of a total 121 points : 84
E_val =  0.6942148760330579


### SVM

In [105]:
y_result = trainSVM(X_train_revenue, y_train_label, X_val_revenue)

In [78]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.38016528925619836
Number of mislabeled points out of a total 121 points : 75
E_val =  0.6198347107438017


### GuassianNB

In [81]:
y_result = trainGussianNB(X_train_revenue, y_train_label, X_val_revenue)

In [82]:
computeAccuracy(X_val_revenue, y_val_label, y_result)

Accuracy = 0.30578512396694213
Number of mislabeled points out of a total 121 points : 84
E_val =  0.6942148760330579


## Test

In [ ]:
isCan = trainLogisticRegression(X_train, y_train_isCan, X_test)
